In [125]:
import os
import re
import io

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

from termcolor import colored
#Define the folder in which all the pdfs are in 
# - assuming the type of analysis needs to be different depending on the journal / time period

#baseFolder = '/home/ishi/Desktop/franzosi/Language-of-Science/articles/AJS pdf files/1946to1966'
#baseFolder = '/Users/saranmedical-smile/Desktop/Language-of-Science/articles/AJS pdf files/1946to1966'
baseFolder = '/Users/saranmedical-smile/Desktop/Language-of-Science/corpus/AJS_1946to1966'
filePaths = os.listdir(baseFolder) # list of all files
filePaths.sort()

In [126]:
filePaths

['AJS_1946_51_4_Antipathy and Social Behavior.txt',
 'AJS_1946_51_4_Definitions of Economic Terms in the Dictionary of Sociology.txt',
 'AJS_1946_51_4_Ethnic Behavior in Industry-Sponsorship and Rejection in a New England Factory.txt',
 'AJS_1946_51_4_Intermarriages Between German Pioneers and Other Nationalities in Minnesota in 1860 and 1870.txt',
 'AJS_1946_51_4_Methods of Measurement of Aesthetic Folkways.txt',
 'AJS_1946_51_4_Patients and Prejudice-Lay Attitudes Toward Women Physicians.txt',
 'AJS_1946_51_4_Social Class and Friendship Among School Children.txt',
 'AJS_1946_51_4_Sociology and the Atom.txt',
 'AJS_1951_56_4_Georg Simmel and Totalitarian Integration.txt',
 'AJS_1951_56_4_Institutional Vulnerability in Mass Society.txt',
 'AJS_1951_56_4_Job-Seeking and the Readjustment Allowance for Veterans.txt',
 'AJS_1951_56_4_Population Private Ownership Technology and the Standard of Living.txt',
 'AJS_1951_56_4_Types of Integration and Their Measurement.txt',
 'AJS_1951_56_4_Witc

In [120]:
def PDFtoString(filePath):
    
    out = io.StringIO()
    with open(os.path.join(baseFolder,filePath), 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, out, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    #string = out.getvalue()
    return out.getvalue() 
string = PDFtoString(os.path.join(baseFolder, filePaths[0]))

In [127]:
with open(os.path.join(baseFolder, filePaths[0]), 'r') as f:
    data = f.read()
print(data) # The header looks terrible - might make sense to use pdf2text just to extract header info

Pale AM ee ee ee at eta
are now only accessible on
the Chicago Journals website at

 

EVR LeU
Antipathy and Social Behavior

Author(s): Chester Alexander

Source: American Journal of Sociology, Vol. 51, N
Published by: The University of Chicago Press
Stable URL: http://www.jstor.org/stable/2770495
Accessed: 06-06-2016 18:17 UTC
Interaction, the basic trait of human so-
cial life, is a complex of interrelationships.
It is considered to be divisible into two ma-
jor parts, namely, that which is closely re-
lated to the biological nature of man and
that which concerns the multiplicity of ac-
tivities which constitute cultural patterns.
It is, of course, recognized that these have
not yet been clearly delineated. While one
might use either of them as an approach to
a study of society, one might also start by
observing human behavior and thus build
up classes of acts according to their similari-
ties. This is not necessarily in disharmony
with the traditional methods; it could be
used to c

### Regex Method 1 - re.compile.finditer(string)

This returns the indices at which the pattern occurs

In [99]:
pattern = re.compile(r'\d[ ]? \w') #Important to start regex expression with r --> raw string
matches = pattern.finditer(string)
for match in matches:
    pass#print(match)

### Regex Method 2 - re.findall(pattern, string)

This returns the pattern itself

In [100]:
%%time
#re.findall(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', string)
for indx, file in enumerate(filePaths):
    print(colored(indx, 'green'), colored(file, 'red'))
    string = PDFtoString(os.path.join(baseFolder, file))
    print(colored(re.findall(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', string)[0], 'blue'))

0 AJS_1946_51_4_Antipathy and Social Behavior.pdf
Accessed: 06-06-2016 18:17 UTC
1 AJS_1946_51_4_Definitions of Economic Terms in the Dictionary of Sociology.pdf
Accessed: 06-06-2016 18:25 UTC
2 AJS_1946_51_4_Ethnic Behavior in Industry-Sponsorship and Rejection in a New England Factory.pdf
Accessed: 06-06-2016 18:19 UTC
3 AJS_1946_51_4_Intermarriages Between German Pioneers and Other Nationalities in Minnesota in 1860 and 1870.pdf
Accessed: 06-06-2016 18:20 UTC
4 AJS_1946_51_4_Methods of Measurement of Aesthetic Folkways.pdf
Accessed: 06-06-2016 18:15 UTC
5 AJS_1946_51_4_Patients and Prejudice-Lay Attitudes Toward Women Physicians.pdf
Accessed: 06-06-2016 18:16 UTC
6 AJS_1946_51_4_Social Class and Friendship Among School Children.pdf
Accessed: 06-06-2016 18:21 UTC
7 AJS_1946_51_4_Sociology and the Atom.pdf
Accessed: 06-06-2016 18:13 UTC
8 AJS_1951_56_4_Georg Simmel and Totalitarian Integration.pdf
Accessed: 06-06-2016 18:33 UTC
9 AJS_1951_56_4_Institutional Vulnerability in Mass Socie

### Split string by re.compile(pattern, string) return val

In [76]:
access_ts = r'(Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC)'
#JSTOR_msg = r'\n \nREFERENCES \nLinked references are available on JSTOR for this article:\nhttp://www.jstor.org/stable/2774936?seq=1&cid=pdf-reference#references_tab_contents \nYou may need to log in to JSTOR to access the linked references.\n \nYour use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at\n\nhttp://about.jstor.org/terms\n\n \n\nJSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of content in a trusted\n\ndigital archive. We use information technology and tools to increase productivity and facilitate new forms of scholarship. For more information about\n\nJSTOR, please contact support@jstor.org.\n\nThe University of Chicago Press is collaborating with JSTOR to digitize, preserve and extend access to\nAmerican Journal of Sociology\n\n'
JSTOR_msg = r'[A-Za-z]*, [0-9]* [A-Za-z]* [0-9]* [0-9]*:[0-9][0-9]:[0-9][0-9] UTC'
str_split = re.split(access_ts, string)
# 0 --> Header w/out access date; 1 --> access_ts; 2 --> Body

In [85]:
matches = re.finditer(JSTOR_msg, str_split[2])
for match in matches:
    print(match)

<re.Match object; span=(659, 688), match='Mon, 06 Jun 2016 18:17:47 UTC'>
<re.Match object; span=(4655, 4684), match='Mon, 06 Jun 2016 18:17:47 UTC'>
<re.Match object; span=(9133, 9162), match='Mon, 06 Jun 2016 18:17:47 UTC'>
<re.Match object; span=(13622, 13651), match='Mon, 06 Jun 2016 18:17:47 UTC'>
<re.Match object; span=(18149, 18178), match='Mon, 06 Jun 2016 18:17:47 UTC'>
<re.Match object; span=(21684, 21713), match='Mon, 06 Jun 2016 18:17:47 UTC'>


In [95]:
JSTOR_msg = r'[A-Za-z]*,\s*[0-9]*\s*[A-Za-z]*\s*[0-9]*\s*[0-9][0-9]:[0-9][0-9]:[0-9][0-9]\s*UTC\s*'
matches = re.finditer(JSTOR_msg, str_split[2])
for indx, match in enumerate(matches):
    print(indx, match)

0 <re.Match object; span=(659, 689), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>
1 <re.Match object; span=(4655, 4685), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>
2 <re.Match object; span=(9133, 9163), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>
3 <re.Match object; span=(13622, 13652), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>
4 <re.Match object; span=(18149, 18179), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>
5 <re.Match object; span=(21684, 21714), match='Mon, 06 Jun 2016 18:17:47 UTC\n'>


In [131]:
re.search(r'ABSTRACT', string)

<re.Match object; span=(1064, 1072), match='ABSTRACT'>